In [ ]:
import os
import math

# Subir un nivel respecto a la carpeta actual
os.chdir("..")

   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 914.9/914.9 kB 8.9 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 20.1 MB/s  0:00:00

   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from Utils import load_parameters_file, from_matrix_to_preset, MSE, denormalize_preset, normalize_preset, pretty_print, PARAM_NAMES, from_preset_to_matrix, mel_spectrogram, mfcc, spectrogram
from Synth import Synth
import numpy as np
import cma
import matplotlib.pyplot as plt
from multiprocessing import Pool
from parallelEvaluation import evaluate_presets
from globals import SAMPLE_RATE, DURATION, PROCESSORS
from IPython.display import Audio
import ipywidgets as widgets
from IPython.display import display

In [3]:
def render_presets(presets):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=presets
    )
    return synth.process_audio()

def evaluate_target(preset):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=preset
    )
    audio = synth.process_audio().astype(np.float64)
    # return mfcc(audio, sr=SAMPLE_RATE)
    return (spectrogram(audio),0, audio )

In [4]:
target_audio = render_presets(load_parameters_file('target.json'))
Audio(target_audio[0], rate=SAMPLE_RATE)

In [ ]:
target_C = evaluate_target(load_parameters_file('target.json'))

def get_fitness(solutions, pool):
    solutions_splitted = np.split(solutions, PROCESSORS)
    presets_splitted = [(denormalize_preset(from_matrix_to_preset(chunk)), target_C) for chunk in solutions_splitted]
    
    solutions_evaluated = pool.map(evaluate_presets, presets_splitted)
    return np.concatenate(solutions_evaluated)

# def evaluate_single(x):
#     """Función que evalúa un solo individuo (requerida por NoiseHandler)."""
#     # print(x)
#     preset = denormalize_preset(from_matrix_to_preset(np.array([x])))
#     result = evaluate_presets((preset, target_C))
#     return (result,)

run = True

boton = widgets.Button(
    description="¡Haz clic aquí!",  # Texto del botón
    button_style='success',         # 'success', 'info', 'warning', 'danger' o ''
    tooltip="Este es un botón",     # Texto que aparece al pasar el ratón
    icon="check"                    # Icono (de FontAwesome)
)

if __name__ == '__main__':
    # target_mfcc = evaluate_presets(load_parameters_file('target.json'))
    # init_solution_mask = np.squeeze(from_preset_to_matrix(normalize_preset(load_parameters_file('init.json'))))

    optimizable_params = PARAM_NAMES

    bestever = cma.optimization_tools.BestSolution()

    with Pool(PROCESSORS) as pool:
        for lam in [15, 30, 60, 120, 240]:

        # Crear estrategia
            x0 = np.zeros(len(optimizable_params))
            sigma0 = 0.3

            options = {
                'popsize': lam,
                'verb_append': bestever.evalsall,
                'bounds': [np.zeros_like(x0), np.ones_like(x0)],
                'tolfun':1e-6
            }
            
            if not math.isinf(bestever.f):
                x0 = bestever.get()[0]

            es = cma.CMAEvolutionStrategy(x0, sigma0, options)

            nh = cma.NoiseHandler(N=len(x0))  # N = número de variables

            gen = 1

            while not es.stop():
                # Generar población
                solutions = np.array(es.ask())  # devuelve una lista de individuos

                # solutions_mask = np.tile(init_solution_mask, (solutions.shape[0], 1)) # Para logica de optimizacion por partes
                # for i, param in enumerate(optimizable_params):
                #     idx = PARAM_NAMES.index(param)
                #     solutions_mask[:,idx] = solutions[:,i]

                fitnesses = get_fitness(solutions, pool)

                best_idx = np.argmin(fitnesses)          # índice del mejor fitness
                best_solution = solutions[best_idx]      # solución correspondiente
                best_fitness = fitnesses[best_idx]       # fitness correspondiente

                print("Gen", gen, "Mejor fitness:", best_fitness, "Sigma", es.sigma)

                # es.sigma *= nh(solutions, fitnesses, evaluate_single, es.ask)

                es.tell(solutions, fitnesses)  # pasar fitness al algoritmo

                gen += 1

                if not run:
                    break

            print('termination:', es.stop())
            cma.s.pprint(es.best.__dict__)

            bestever.update(es.best)

            if not run:
                break

            if bestever.f < 0.0082:
                break

            if bestever.f < 1e-5:  # si se alcanza el óptimo global
                break             # detener todas las corridas

    # Mejor solución
    best_solution = bestever.get()[0]
    print("Mejor individuo:", best_solution)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 42 (seed=578682, Thu Jan 15 13:54:18 2026)
Gen 1 Mejor fitness: 0.016583811537755944 Sigma 0.3
Gen 2 Mejor fitness: 0.01567942436155813 Sigma 0.28035160818088894
Gen 3 Mejor fitness: 0.01883933248607003 Sigma 0.2677071365121127
Gen 4 Mejor fitness: 0.018046362514415448 Sigma 0.2583570403835263
Gen 5 Mejor fitness: 0.016279359254559176 Sigma 0.24970840701097985
Gen 6 Mejor fitness: 0.016139404693194248 Sigma 0.24713533672588445
Gen 7 Mejor fitness: 0.018973693342040892 Sigma 0.24704792068247997
Gen 8 Mejor fitness: 0.020622183626842587 Sigma 0.2485078961000804
Gen 9 Mejor fitness: 0.025502010141612737 Sigma 0.2512515438570144
Gen 10 Mejor fitness: 0.021736650687472592 Sigma 0.2503633570783962
Gen 11 Mejor fitness: 0.020443298696448047 Sigma 0.24962745526191296
Gen 12 Mejor fitness: 0.017283139054726657 Sigma 0.24649023963777225
Gen 13 Mejor fitness: 0.014837795861664687 Sigma 0.24136086643077867
Gen 14 Mejor fitness: 0.019717808358879665 

KeyboardInterrupt: 

In [ ]:
# from deap import base, creator, tools, algorithms
# import array
# import random

# IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY = len(PARAM_NAMES), 0, 1, 0, 1

# def checkStrategy(minstrategy):
#     def decorator(func):
#         def wrappper(*args, **kargs):
#             children = func(*args, **kargs)
#             for child in children:
#                 for i, s in enumerate(child.strategy):
#                     if s < minstrategy:
#                         child.strategy[i] = minstrategy
#             return children
#         return wrappper
#     return decorator

# def checkBounds(minval, maxval):
#     def decorator(func):
#         def wrapper(*args, **kwargs):
#             children = func(*args, **kwargs)
#             for child in children:
#                 for i in range(len(child)):
#                     if child[i] < minval:
#                         child[i] = minval
#                     elif child[i] > maxval:
#                         child[i] = maxval
#             return children
#         return wrapper
#     return decorator

# creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
# creator.create("Strategy", array.array, typecode="d")

# def generateES(icls, scls, size, imin, imax, smin, smax):
#     ind = icls(random.uniform(imin, imax) for _ in range(size))
#     ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
#     return ind

# toolbox = base.Toolbox()

# toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
#     IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# toolbox.register("evaluate", evaluate_single)

# toolbox.register("mate", tools.cxESBlend, alpha=0.1)
# toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
# toolbox.register("select", tools.selTournament, tournsize=3)

# toolbox.decorate("mate", checkStrategy(MIN_STRATEGY))
# toolbox.decorate("mutate", checkStrategy(MIN_STRATEGY))

# toolbox.decorate("mate", checkBounds(MIN_VALUE, MAX_VALUE))
# toolbox.decorate("mutate", checkBounds(MIN_VALUE, MAX_VALUE))

# def main():
#     MU, LAMBDA = 10, 100
#     pop = toolbox.population(n=MU)
#     hof = tools.HallOfFame(1)
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     # stats.register("avg", np.mean)
#     # stats.register("std", np.std)
#     # stats.register("min", np.min)
#     stats.register("max", np.max)

#     pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
#         cxpb=0.6, mutpb=0.3, ngen=500, stats=stats, halloffame=hof, verbose=True)

#     # return pop, logbook, hof
#     print('El mejor individuo es', hof[0], evaluate_single(hof[0]))

# main()

In [ ]:
# solution = init_solution_mask
# for i, param in enumerate(optimizable_params):
#     idx = PARAM_NAMES.index(param)
#     solution[idx] = best_solution[i]

solution = np.expand_dims(best_solution, axis=0)
error = evaluate_presets((denormalize_preset(from_matrix_to_preset(solution)), target_C))
print('Error', error)

target_audio = render_presets(load_parameters_file('target.json'))
predicted_audio = render_presets(denormalize_preset(from_matrix_to_preset(solution)))

Error [0.01680744]


In [ ]:
from scipy.io import wavfile

# NAME = "exports/s3_a8"

# audio_int16 = np.int16(target_audio[0] / np.max(np.abs(target_audio[0])) * 32767)
# wavfile.write(f"{NAME}_target.wav", SAMPLE_RATE, audio_int16)

Audio(target_audio[0], rate=SAMPLE_RATE)

In [ ]:
# audio_int16 = np.int16(predicted_audio[0] / np.max(np.abs(predicted_audio[0])) * 32767)
# wavfile.write(f"{NAME}_predicted_{error}.wav", SAMPLE_RATE, audio_int16)

Audio(predicted_audio[0], rate=SAMPLE_RATE)

In [ ]:
denormalized_predicted_preset = denormalize_preset(from_matrix_to_preset(solution))
pretty_print(denormalized_predicted_preset)

{
  lfo1_rate:   [
    17.256598228108764
  ]
  lfo1_shape:   [
    3.0133224
  ]
  lfo2_rate:   [
    3.2230926335760888
  ]
  lfo2_shape:   [
    3.69577576
  ]
  osc1_shape:   [
    2.93477772
  ]
  osc1_phase:   [
    0.77946573
  ]
  osc1_volume:   [
    0.18740611
  ]
  osc1_freq:   [
    4576.017696530759
  ]
  osc1_vdepth:   [
    0.64518553
  ]
  osc1_pdepth:   [
    0.8003704
  ]
  osc2_shape:   [
    0.78454088
  ]
  osc2_phase:   [
    0.44972754
  ]
  osc2_volume:   [
    0.15077782
  ]
  osc2_freq:   [
    22.29880415216944
  ]
  osc2_vdepth:   [
    0.41158688
  ]
  osc2_pdepth:   [
    0.22116579
  ]
  osc3_shape:   [
    0.24705544
  ]
  osc3_phase:   [
    0.55399495
  ]
  osc3_volume:   [
    0.962387
  ]
  osc3_freq:   [
    21909.589908784455
  ]
  osc3_vdepth:   [
    0.6532194
  ]
  osc3_pdepth:   [
    0.02810401
  ]
  osc4_shape:   [
    0.010512
  ]
  osc4_phase:   [
    0.28933924
  ]
  osc4_volume:   [
    0.10091183
  ]
  osc4_freq:   [
    60.0069542006444

In [ ]:
np.diagonal(es.C)

array([0.94894334, 0.85037441, 1.08138868, 0.67034162, 1.19918752,
       0.88947025, 0.12090519, 1.16905239, 0.8885555 , 0.01954993,
       0.80602447, 2.20433542, 0.80836165, 1.41144238, 0.99985506,
       1.11474639, 0.80022709, 1.04600022, 1.00585479, 1.1301805 ,
       0.96668918, 0.69343029, 0.90741101, 1.2644367 , 0.90225649,
       0.58645467, 1.20649605, 0.54714629, 1.14272979, 1.31448035,
       1.10361058, 0.75211311, 0.83514454, 0.87869128, 1.03076819,
       0.88923977, 0.68670739, 0.58213642, 1.17393113, 1.2196411 ,
       1.07093475, 1.01401017])